In [2]:
import base64
import datetime
import time
import json
import gc
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from numpy import median
import matplotlib.dates as mdates

np.random.seed(1337)
%matplotlib inline
sns.set(font_scale=1.5)
rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
d = pd.read_csv('tmp/dmy_ho_train_ind_recibo_ult1.csv')
d = d.iloc[np.random.permutation(len(d))]
del d['customer_id']
l = d.target.copy()
del d['target']
del d['target_name']

lim_train = len(d)/2
tr = d[:lim_train].copy()
trl = l[:lim_train].copy()
te = d[lim_train:].copy()
tel = l[lim_train:].copy()
del d
del l
gc.collect()

45

In [ ]:
param_grid = {
    'max_depth': [3, 9],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [10, 100],
    'gamma': [0, 0.01]
}

xgb = XGBClassifier()
m = GridSearchCV(
        estimator=xgb,
        param_grid=param_grid,
        cv=2)
m.fit(tr, trl)
print m.best_score_
print m.best_params_

In [10]:
param_grid = {
    'max_depth': [9],
    'learning_rate': [0.01],
    'n_estimators': [10],
    'gamma': [0]
}

xgb = XGBClassifier()
m = GridSearchCV(
        estimator=xgb,
        param_grid=param_grid,
        cv=2)
m.fit(tr, trl)
print m.best_score_
print m.best_params_

0.910595494602
{'n_estimators': 10, 'learning_rate': 0.01, 'max_depth': 9, 'gamma': 0}


In [6]:
products = '''ind_recibo_ult1
ind_cco_fin_ult1
ind_nom_pens_ult1
ind_nomina_ult1
ind_cno_fin_ult1
ind_tjcr_fin_ult1
ind_ecue_fin_ult1
ind_dela_fin_ult1
ind_reca_fin_ult1
ind_ctma_fin_ult1
ind_valo_fin_ult1
ind_fond_fin_ult1
ind_deco_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_plan_fin_ult1
ind_ctju_fin_ult1
ind_deme_fin_ult1
ind_cder_fin_ult1
ind_pres_fin_ult1
ind_hip_fin_ult1
ind_viv_fin_ult1
ind_aval_fin_ult1
ind_ahor_fin_ult1'''.split('\n')

In [12]:
for p in products:
    print '=' * 30
    print 'product:', p
    
    d = pd.read_csv('tmp/dmy_ho_train_{0}.csv'.format(p))
    d = d.iloc[np.random.permutation(len(d))]
    del d['customer_id']
    l = d.target.copy()
    del d['target']
    del d['target_name']

    lim_train = len(d)/2
    tr = d[:lim_train].copy()
    trl = l[:lim_train].copy()
    te = d[lim_train:].copy()
    tel = l[lim_train:].copy()
    del d
    del l
    gc.collect()

    print 'test labels:'
    print tel.value_counts()

    m = XGBClassifier(
            n_estimators=30,
            learning_rate=0.1,
            max_depth=9,
            gamma=0,
            min_child_weight=1,
            scale_pos_weight=10.0)

    m.fit(tr.fillna(0), trl)
    p = m.predict(te.fillna(0))

    print 'ROC AUC score:'
    print roc_auc_score(tel, p)

    cmx = pd.DataFrame(confusion_matrix(tel, p))
    cmx.index.name='actual'
    cmx.columns.name='predicted'
    print 'confusion matrix:'
    print cmx

    del tr
    del trl
    del te
    del tel
    del m
    del p
    gc.collect()

product: ind_recibo_ult1
test labels:
0.0    150983
1.0     29557
Name: target, dtype: int64
ROC AUC score:
0.871472998556
confusion matrix:
predicted       0      1
actual                  
0          116836  34147
1             913  28644
product: ind_cco_fin_ult1
test labels:
0.0    146949
1.0     19937
Name: target, dtype: int64
ROC AUC score:
0.947549809221
confusion matrix:
predicted       0      1
actual                  
0          138116   8833
1             893  19044
product: ind_nom_pens_ult1
test labels:
0.0    148278
1.0     15305
Name: target, dtype: int64
ROC AUC score:
0.910212879928
confusion matrix:
predicted       0      1
actual                  
0          132405  15873
1            1110  14195
product: ind_nomina_ult1
test labels:
0.0    146801
1.0     14915
Name: target, dtype: int64
ROC AUC score:
0.911403436192
confusion matrix:
predicted       0      1
actual                  
0          131655  15146
1            1104  13811
product: ind_cno_fin_ult1
test la

KeyboardInterrupt: 